In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from pathlib import Path
import sqlite3

In [2]:
sset = "XLN"
urlstring ='Ixalan'
releasedate = '2017-09-29'

In [3]:
url = f"https://www.mtggoldfish.com/sets/{urlstring}#paper"
res = requests.get(url)

In [4]:
markup = res.text
delim =  f'var d = "Date,{sset}-main_set";'                      #'hash == "#tab-paper"'
chunks = markup.split(delim)
chunks.pop(0)
markup = ''.join(chunks)
delim = 'g = new Dygraph'
chunks = markup.split(delim)
markup = chunks.pop(0).rstrip()
delim = 'g = new Dygraph'
chunks = markup.split(delim)
markup = chunks.pop(0).rstrip()
lines = markup.splitlines()
for i, line in enumerate(lines):
    lines[i] = line.strip().lstrip('d += "\\n').rstrip('";')
    csv = '\n'.join(lines)
lines.pop(0)
df =[]
for i in lines:
    i = i.split(',')
    df.append([i[0],i[1]])


In [5]:
df = pd.DataFrame(df, columns = ['date', 'price'])
df['setcode'] = sset
df = df.loc[df['date'] >= releasedate] 
df.head(3)

,date,price,setcode
32,2017-09-29,328.94,XLN
33,2017-09-30,326.81,XLN
34,2017-10-01,327.37,XLN


In [7]:
##price =  Path("../Resources/price.db")
con = sqlite3.connect(price)        # conect to price db
cur = con.cursor()

In [8]:
 for row in df.itertuples():           # else not exist write new data to db 
            insert_sql = f''' INSERT INTO 'goldfish' ('date', 'price', 'setcode') VALUES ('{row[1]}',"{row[2]}",'{row[3]}')'''
            cur.execute(insert_sql)

In [9]:
##con.commit()